In [1]:
import DataFunctions
import numpy as np



Overall idea of simulation: Do pairings taking into consideration current score (win-lose rate) and their geographical location. It will use an adaptation of the Swiss Pairing System, the main difference is that instead of participants having a ELO score or rating and being paired by it; we consider an objective function on the total distance travelled and find proper pairings that minimize it. 

Tentative Objective function: $\sum (d_i)^2 + max(d_i)^2$. This function uses L2 norm and a L2 regularization term. We have to consider a regularization term as it helps us to avoid having multiple low distances and 1 large distance. For simplicity we can remove the regularization to trial the algorithm, and then regularization.

Rules per each round:
1. Split teams by their current score (#wins)
2. Pair from highest scoring group to lowest scoring group.
3. Per each group, find the group of pairs that minimize the objective function.
4. If there are odd # of teams, demote unpaired team into next scoring group. If it has been demoted before, use the next possible pairing that includes that team.
5. If there exist unpaired teams, demote them into the next scoring group. 
6. After making all possible pairings, set distance of teams that are playing together to inf.
7. For each pairing, if both teams have the same # of home games, pick one to be home and one to be away randomly. If one team has more # home games than the other, pick the team with the least # home games to be home, the other to be the away.

Things to store in db:
1. Each team 2022 stats.
2. Each team # (from 0 to 131).
3. Simulated season pairings, w-l record and h-a count.
4. Simulated games and scores.

Simulating each pairing:
1. Use 2022 stats to predict a spread and totalpts. We use those as means.
2. Using a normal distribution on the spread and totalpts with the standard deviation found on the model. We randomly generate a result with the simulated spread and totalpts.
3. We use obtained values to create an score.

In [35]:
#creating random distance matrix of size = 8
nteams = 8

m_dist = np.around(np.random.uniform(1,10,size=(nteams,nteams)),decimals=3)

for i in range(nteams):
    for j in range(nteams):
        m_dist[j,i]=m_dist[i,j]

for i in range(nteams):
    m_dist[i,i]=np.inf

m_dist

array([[  inf, 6.252, 7.632, 9.554, 1.522, 3.054, 3.2  , 6.407],
       [6.252,   inf, 9.217, 7.168, 2.391, 9.171, 7.043, 1.717],
       [7.632, 9.217,   inf, 1.147, 9.479, 4.053, 1.919, 5.735],
       [9.554, 7.168, 1.147,   inf, 1.61 , 8.552, 5.376, 2.189],
       [1.522, 2.391, 9.479, 1.61 ,   inf, 2.976, 2.66 , 3.628],
       [3.054, 9.171, 4.053, 8.552, 2.976,   inf, 8.641, 4.297],
       [3.2  , 7.043, 1.919, 5.376, 2.66 , 8.641,   inf, 4.262],
       [6.407, 1.717, 5.735, 2.189, 3.628, 4.297, 4.262,   inf]])

In [49]:
#storing wins per team.
curr_data = np.zeros(shape=(nteams,2))
curr_data[:,0] = np.arange(nteams)
curr_data

array([[0., 0.],
       [1., 0.],
       [2., 0.],
       [3., 0.],
       [4., 0.],
       [5., 0.],
       [6., 0.],
       [7., 0.]])

In [98]:
#splitting in groups
sim1group0 = np.array([[x[0].astype(int),0] for x in curr_data if x[1]==0])
# sim1group1 = [[x[0].astype(int),0] for x in curr_data if x[1]==1]
sim1group0

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0],
       [6, 0],
       [7, 0]])

We cannot find all pairs and run for all of them. That operation has time complexity O(n!) and for 130ish teams is not feasible as it is about $10^{220}$. We need to use another approach.<br>
Fastest approach is done by sorting all possible distances, then picking the suitable one and remove unsuitables from the list. We only need to sort once since at the beginning all teams have to play. We should also count the # of possible matches and use that to do the sorting as well.<br><br>

Algorithm procedure:
1. Sort all distances
2. Split in groups
3. Sort per number of inf. Descending order.
4. If team can play with another team, pick smallest distance.
5. If team can't play with another team, they get demoted. 

In [104]:
def checkinf(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if(m_dist[i,j]==np.inf and i!=j):
                a[i,1]+=1
    return a

In [105]:
checkinf(sim1group0)

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0],
       [6, 0],
       [7, 0]])

In [110]:
L = []
for i in range(len(sim1group0)):
    for j in range(len(sim1group0)):
        if(i<j and m_dist[i,j]!=np.inf):
            L.append((-np.max(sim1group0[i,1],sim1group0[j,1]),m_dist[i,j],i,j))

LS = sorted(L)

In [111]:
LS

[(0, 1.147, 2, 3),
 (0, 1.522, 0, 4),
 (0, 1.61, 3, 4),
 (0, 1.717, 1, 7),
 (0, 1.919, 2, 6),
 (0, 2.189, 3, 7),
 (0, 2.391, 1, 4),
 (0, 2.66, 4, 6),
 (0, 2.976, 4, 5),
 (0, 3.054, 0, 5),
 (0, 3.2, 0, 6),
 (0, 3.628, 4, 7),
 (0, 4.053, 2, 5),
 (0, 4.262, 6, 7),
 (0, 4.297, 5, 7),
 (0, 5.376, 3, 6),
 (0, 5.735, 2, 7),
 (0, 6.252, 0, 1),
 (0, 6.407, 0, 7),
 (0, 7.043, 1, 6),
 (0, 7.168, 1, 3),
 (0, 7.632, 0, 2),
 (0, 8.552, 3, 5),
 (0, 8.641, 5, 6),
 (0, 9.171, 1, 5),
 (0, 9.217, 1, 2),
 (0, 9.479, 2, 4),
 (0, 9.554, 0, 3)]

In [103]:
def selection(LS,group):
    s = set(group[:,0])
    return s

In [102]:
selection(LS=LS,group=sim1group0)

{0, 1, 2, 3, 4, 5, 6, 7}